# Entrenamiento de modelos básicos con los datos de Santa Maria y Stanford

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import random
import csv
import os
from pathlib import Path
from enum import Enum
from sklearn.metrics import roc_auc_score, recall_score, precision_score
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from scipy.ndimage import zoom
from sklearn.utils.class_weight import compute_class_weight
from simple import SimpleModel, SimpleModel2, SimpleCNNModel
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras import regularizers

/home/roberto/anaconda3/envs/lung_radiomics/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def show_slice_window(slice, level, window):

    """
    Permite ajustar nivel y ancho de ventana para mejorar contraste de la imagen.
    input: imagen np.array 2D.
    output: imagen np.array 2D ajustada.
   """
    max = level + window/2
    min = level - window/2
    return slice.clip(min,max)

def roiExtractionResize(img_exam, mask_exam, img_size, margin=3):
    img_instance, mask_instance = img_exam.numpy(), mask_exam.numpy()
    index = np.where(mask_instance)

    x1, x2 = np.unique(index[0])[0], np.unique(index[0])[-1]
    y1, y2 = np.unique(index[1])[0], np.unique(index[1])[-1]

    width, height = x2 - x1, y2 - y1
    lmargin = (width - height) // 2
    rmargin = (width - height) - lmargin
    tmargin = (height - width) // 2
    bmargin = (height - width) - tmargin

    x1, x2 = x1 - tmargin, x2 + bmargin
    y1, y2 = y1 - lmargin, y2 + rmargin

    roi = img_instance[x1-margin:x2+margin, y1-margin:y2+margin]

    # Normalize the ROI to [0, 1]
    roi_normalized = roi # (roi - np.min(roi)) / (np.max(roi) - np.min(roi))

    # Resize the image using NumPy
    roi_resized = zoom(roi_normalized, (img_size / roi_normalized.shape[0], img_size / roi_normalized.shape[1]), order=1)

    return roi_resized


def roiExtractionSize(img, mask, total_size=None, margin=3):
    """
    Function to extract ROIs from images while ensuring a consistent total size for all ROIs.

    INPUT:
    img: Numpy array of images.
    mask: Numpy array of masks.
    total_size: The desired total size (width and height) of the extracted ROIs.

    OUTPUT: Numpy array containing the ROIs.
    """
    
    img_instance, mask_instance = img.numpy(), mask.numpy()
    index = np.where(mask_instance)

    if total_size == None:
        roi = img_instance[np.unique(index[0])[0]-margin:np.unique(index[0])[-1]+margin, np.unique(index[1])[0]-margin: np.unique(index[1])[-1]+margin]
    else:
    
        # Calculate the center of the mask.
        center_row, center_col = int(np.mean(index[0])), int(np.mean(index[1]))
    
        # Calculate the size of the ROI based on the total size.
        half_size = total_size // 2
    
        # Determine ROI boundaries with the margin.
        min_row = max(0, center_row - half_size)
        max_row = min(mask_instance.shape[0], center_row + half_size)
        min_col = max(0, center_col - half_size)
        max_col = min(mask_instance.shape[1], center_col + half_size)
    
        # Calculate the width and height of the ROI.
        roi_height = max_row - min_row
        roi_width = max_col - min_col
    
        # Case 1: If the ROI is smaller than the total_size, add a margin to make it total_size.
        if roi_height < total_size:
            margin = (total_size - roi_height) // 2
            min_row -= margin
            max_row += margin
    
        if roi_width < total_size:
            margin = (total_size - roi_width) // 2
            min_col -= margin
            max_col += margin
    
        # Case 2: If the ROI is larger than total_size, resize it.
        if roi_height > total_size or roi_width > total_size:
            scale_factor = total_size / max(roi_height, roi_width)
            new_height = int(roi_height * scale_factor)
            new_width = int(roi_width * scale_factor)
            min_row = max(center_row - new_height // 2, 0)
            max_row = min(min_row + new_height, mask_instance.shape[0])
            min_col = max(center_col - new_width // 2, 0)
            max_col = min(min_col + new_width, mask_instance.shape[1])
    
        # Extract the ROI with the desired size.
        roi = img_instance[min_row:max_row, min_col:max_col]
    
    return roi

## Implementación K-Fold para conjuntos de datos de Santa María y Stanford
### 1. Stanford Stratified Kfold

In [3]:
# stanford datasets - ct, pet, torax3d
class StanfordDatasetType(Enum):
    CT = 'ct'
    PET = 'pet'
    TORAX3D = 'chest_ct'

def get_stratified_stanford_kfold_datasets(ds_type, img_size=32, k=5, random_seed=None):
    '''Creates stratified k-fold cross-validation datasets with balanced class distribution.
    Returns a list of k pairs of training and testing datasets with balanced classes.'''

    if ds_type not in StanfordDatasetType._value2member_map_:
        raise ValueError("Invalid dataset type. Use one of: ct, pet, torax3d")

    stanford_dataset, stanford_info = tfds.load(f'stanford_dataset/{ds_type}', with_info=True)
    
    # Set the random seed
    random.seed(random_seed)

    pos_patients = []
    neg_patients = []
    
    stanford_csv_file = Path('stanford_info_csv.csv')
    with stanford_csv_file.open() as f:
      for row in csv.DictReader(f):
        if row['EGFR mutation status'] == 'Mutant':
          pos_patients.append(row['Case ID'])
        else:
          neg_patients.append(row['Case ID'])

    # Get the split keys (splits) of the dataset
    split_keys = list(stanford_info.splits.keys())

    # Find positive and negative patients that are also in the split keys
    pos_patients = [patient for patient in pos_patients if patient in split_keys]
    neg_patients = [patient for patient in neg_patients if patient in split_keys]

    # Shuffle the order of positive and negative patients for randomness
    random.shuffle(pos_patients)
    random.shuffle(neg_patients)

    # Create labels for patients (1 for positive, 0 for negative)
    labels = [1] * len(pos_patients) + [0] * len(neg_patients)
    patients = pos_patients + neg_patients
    # Initialize stratified k-fold cross-validator
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_seed)

    fold_datasets = []
    
    for train_indices, test_indices in skf.split(patients, labels):
        training_patients = [patients[i] for i in train_indices]
        testing_patients = [patients[i] for i in test_indices]

        # Create dictionaries to hold the training and testing data
        training_data = {patient: stanford_dataset[patient] for patient in training_patients}
        testing_data = {patient: stanford_dataset[patient] for patient in testing_patients}

        # Create a generator for the training dataset
        def generate_training_data():
            for patient_id in training_patients:
                patient_data = training_data[patient_id]
                for data in patient_data:
                    mask_exam = data['mask_exam']
                    img_exam = data['img_exam']
                    roi_extract = roiExtractionSize(img_exam, mask_exam, img_size, margin=8)
                    roi_extract = show_slice_window(roi_extract, 60, 120)
                    
                    roi_extract = roi_extract[:,:,np.newaxis]
                    # Convert grayscale image (1 channel) to RGB (3 channels)
                    #roi_extract_rgb = np.stack((roi_extract,) * 3, axis=2).astype(np.float32)
                    yield roi_extract, data['label']

        # Create a TensorFlow Dataset from the generator
        training_dataset = tf.data.Dataset.from_generator(
            generate_training_data,
            output_signature=(
                tf.TensorSpec(shape=(img_size, img_size, 1), dtype=tf.float32, name="imagen"),  # For mask_exam
                tf.TensorSpec(shape=(), dtype=tf.int64, name="label")  # For label
            )
        )

        # Create a generator for the testing dataset
        def generate_testing_data():
            for patient_id in testing_patients:
                patient_data = testing_data[patient_id]
                for data in patient_data:
                    mask_exam = data['mask_exam']
                    img_exam = data['img_exam']
                    roi_extract = roiExtractionSize(img_exam, mask_exam, img_size, margin=8)
                    roi_extract = show_slice_window(roi_extract, 60, 120)
                    
                    roi_extract = roi_extract[:,:,np.newaxis]
                    # Convert grayscale image (1 channel) to RGB (3 channels)
                    #roi_extract_rgb = np.stack((roi_extract,) * 3, axis=2).astype(np.float32)
                    yield roi_extract, data['label']

        # Create a TensorFlow Dataset from the generator
        testing_dataset = tf.data.Dataset.from_generator(
            generate_testing_data,
            output_signature=(
                tf.TensorSpec(shape=(img_size, img_size, 1), dtype=tf.float32, name="imagen"),  # For mask_exam
                tf.TensorSpec(shape=(), dtype=tf.int64, name="label")  # For label
            )
        )

        fold_datasets.append((training_dataset, testing_dataset))

    return fold_datasets

In [4]:
stanford_kfold_ds = get_stratified_stanford_kfold_datasets('ct') # pet, torax3d, body
batch_sz = 5

initial_learning_rate = 0.001
decay_steps = 100
alpha = 0.0

# Initialize lists to store test loss, accuracy, and AUC for each fold
test_losses = []
test_accuracies = []
test_precisions = []
test_recalls = []
test_roc_aucs = []

# Initialize EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Assuming you have calculated class weights using some method
#class_weights = {0: 1.0, 1: 5.0}  # Adjust the values based on your class distribution


for i, (training_ds, testing_ds) in enumerate(stanford_kfold_ds):
    training_ds = training_ds.batch(batch_sz)
    testing_ds = testing_ds.batch(batch_sz)
    
    print(f"Fold: {i}")
    
    # Create and compile your model for this fold
    #model = SimpleModel2(1)
    # Example usage:
    num_classes = 1  # For binary classification
    model = SimpleModel2(num_classes)
    model.compile(
        #optimizer=AdamW(learning_rate=CosineDecay(initial_learning_rate, decay_steps, alpha=alpha), 
                        #weight_decay=0.001),
        optimizer='adam',
        loss='binary_crossentropy', 
        metrics=['accuracy', AUC(name='auc', curve='PR'), Recall(), Precision()]
    )
    
    # Fit the model with early stopping
    model.fit(training_ds, epochs=10, validation_data=testing_ds, callbacks=[early_stopping])
    
    # Evaluate the model on the testing dataset for this fold
    results = model.evaluate(testing_ds)

    # Save test loss, accuracy, and AUC for this fold
    test_losses.append(results[0])
    test_accuracies.append(results[1])
    test_roc_aucs.append(results[2])
    test_recalls.append(results[3])
    test_precisions.append(results[4])
    
    print(f"Test accuracy for Fold: {results[1]}")
    print(f"AUC score for Fold: {results[2]}")
    print(f"Recall for Fold: {results[3]}")
    print(f"Precision for Fold: {results[4]}")

# Calculate and print the mean metrics
mean_test_loss = np.mean(test_losses)
mean_test_accuracy = np.mean(test_accuracies)
mean_test_auc = np.mean(test_roc_aucs)
mean_test_recall = np.mean(test_recalls)
mean_test_precision = np.mean(test_precisions)

print(f"\nMean Test Loss: {mean_test_loss}")
print(f"Mean Test Accuracy: {mean_test_accuracy}")
print(f"Mean AUC Score: {mean_test_auc}")
print(f"Mean Recall: {mean_test_recall}")
print(f"Mean Precision: {mean_test_precision}")


Fold: 0
Epoch 1/10
257/257 [==============================] - 63s 226ms/step - loss: 3.5427 - accuracy: 0.8654 - auc: 0.5837 - recall: 0.8818 - precision: 0.5689 - val_loss: 2.8128 - val_accuracy: 0.8218 - val_auc: 0.2141 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
257/257 [==============================] - 48s 186ms/step - loss: 1.9368 - accuracy: 0.8000 - auc: 0.3374 - recall: 0.3591 - precision: 0.4051 - val_loss: 1.6919 - val_accuracy: 0.8218 - val_auc: 0.3263 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
257/257 [==============================] - 43s 166ms/step - loss: 1.1814 - accuracy: 0.7930 - auc: 0.2971 - recall: 0.3045 - precision: 0.3722 - val_loss: 1.1963 - val_accuracy: 0.8218 - val_auc: 0.2515 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
257/257 [==============================] - 51s 196ms/step - loss: 0.8642 - accuracy: 0.7930 - auc: 0.3223 - recall: 0.3182 - precision: 0.3763 - val_loss: 0.9769 - val_accuracy: 0.

KeyboardInterrupt: 

In [5]:
# stanford datasets - ct, pet, torax3d
class SantaMariaDatasetType(Enum):
    CT = 'body'
    PET = 'pet'
    TORAX3D = 'torax3d'

def get_stratified_santamaria_kfold_datasets(ds_type, img_size=32, k=5, random_seed=None):
    '''Creates stratified k-fold cross-validation datasets with balanced class distribution.
    Returns a list of k pairs of training and testing datasets with balanced classes.'''

    if ds_type not in SantaMariaDatasetType._value2member_map_:
        raise ValueError("Invalid dataset type. Use one of: ct, pet, torax3d")

    santamaria_ds, santamaria_info = tfds.load(f'santa_maria_dataset/{ds_type}', with_info=True)
    
    # Set the random seed
    random.seed(random_seed)

    # Define the positive and negative patients
    pos_patients = [f'sm_{str(i).zfill(3)}' for i in range(1, 13)]
    neg_patients = [f'sm_{str(i).zfill(3)}' for i in range(13, 36)]

    # Get the split keys (splits) of the dataset
    split_keys = list(santamaria_info.splits.keys())

    # Find positive and negative patients that are also in the split keys
    pos_patients = [patient for patient in pos_patients if patient in split_keys]
    neg_patients = [patient for patient in neg_patients if patient in split_keys]

    # Shuffle the order of positive and negative patients for randomness
    random.shuffle(pos_patients)
    random.shuffle(neg_patients)

    # Create labels for patients (1 for positive, 0 for negative)
    labels = [1] * len(pos_patients) + [0] * len(neg_patients)
    patients = pos_patients + neg_patients
    # Initialize stratified k-fold cross-validator
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_seed)

    fold_datasets = []
    
    for train_indices, test_indices in skf.split(patients, labels):
        training_patients = [patients[i] for i in train_indices]
        testing_patients = [patients[i] for i in test_indices]

        # Create dictionaries to hold the training and testing data
        training_data = {patient: santamaria_ds[patient] for patient in training_patients}
        testing_data = {patient: santamaria_ds[patient] for patient in testing_patients}

        # Create a generator for the training dataset
        def generate_training_data():
            for patient_id in training_patients:
                patient_data = training_data[patient_id]
                for data in patient_data:
                    mask_exam = data['mask_exam']
                    img_exam = data['img_exam']
                    roi_extract = roiExtractionSize(img_exam, mask_exam, img_size, margin=8)
                    roi_extract = show_slice_window(roi_extract, 60, 120)
                    
                    roi_extract = roi_extract[:,:,np.newaxis]
                    # Convert grayscale image (1 channel) to RGB (3 channels)
                    #roi_extract_rgb = np.stack((roi_extract,) * 3, axis=2).astype(np.float32)
                    yield roi_extract, data['label']

        # Create a TensorFlow Dataset from the generator
        training_dataset = tf.data.Dataset.from_generator(
            generate_training_data,
            output_signature=(
                tf.TensorSpec(shape=(img_size, img_size, 1), dtype=tf.float32, name="imagen"),  # For mask_exam
                tf.TensorSpec(shape=(), dtype=tf.int64, name="label")  # For label
            )
        )

        # Create a generator for the testing dataset
        def generate_testing_data():
            for patient_id in testing_patients:
                patient_data = testing_data[patient_id]
                for data in patient_data:
                    mask_exam = data['mask_exam']
                    img_exam = data['img_exam']
                    roi_extract = roiExtractionSize(img_exam, mask_exam, img_size, margin=8)
                    roi_extract = show_slice_window(roi_extract, 60, 120)
                    
                    roi_extract = roi_extract[:,:,np.newaxis]
                    # Convert grayscale image (1 channel) to RGB (3 channels)
                    #roi_extract_rgb = np.stack((roi_extract,) * 3, axis=2).astype(np.float32)
                    yield roi_extract, data['label']

        # Create a TensorFlow Dataset from the generator
        testing_dataset = tf.data.Dataset.from_generator(
            generate_testing_data,
            output_signature=(
                tf.TensorSpec(shape=(img_size, img_size, 1), dtype=tf.float32, name="imagen"),  # For mask_exam
                tf.TensorSpec(shape=(), dtype=tf.int64, name="label")  # For label
            )
        )

        fold_datasets.append((training_dataset, testing_dataset))

    return fold_datasets


In [6]:
santamaria_kfold_ds = get_stratified_santamaria_kfold_datasets('body') # pet, torax3d, body
batch_sz = 5

initial_learning_rate = 0.001
decay_steps = 100
alpha = 0.0

# Initialize lists to store test loss, accuracy, and AUC for each fold
test_losses = []
test_accuracies = []
test_precisions = []
test_recalls = []
test_aucs = []

# Initialize EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

for i, (training_ds, testing_ds) in enumerate(santamaria_kfold_ds):
    training_ds = training_ds.batch(batch_sz)
    testing_ds = testing_ds.batch(batch_sz)
    
    print(f"Fold: {i}")
    
    # Create and compile your model for this fold
    model = SimpleModel2(1)
    model.compile(
        'adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy', AUC(name='auc', curve='PR'), Recall(), Precision()]
    )
    
    model.fit(training_ds, epochs=10, validation_data=testing_ds)
    
    # Evaluate the model on the testing dataset for this fold
    results = model.evaluate(testing_ds)

    # Save test loss, accuracy, and AUC for this fold
    test_losses.append(results[0])
    test_accuracies.append(results[1])
    test_aucs.append(results[2])
    test_recalls.append(results[3])
    test_precisions.append(results[4])
    
    print(f"Test accuracy for Fold: {results[1]}")
    print(f"AUC score for Fold: {results[2]}")
    print(f"Recall for Fold: {results[3]}")
    print(f"Precision for Fold: {results[4]}")

# Calculate and print the mean metrics
mean_test_loss = np.mean(test_losses)
mean_test_accuracy = np.mean(test_accuracies)
mean_test_auc = np.mean(test_aucs)
mean_test_recall = np.mean(test_recalls)
mean_test_precision = np.mean(test_precisions)

print(f"\nMean Test Loss: {mean_test_loss}")
print(f"Mean Test Accuracy: {mean_test_accuracy}")
print(f"Mean AUC Score: {mean_test_auc}")
print(f"Mean Recall: {mean_test_recall}")
print(f"Mean Precision: {mean_test_precision}")

Fold: 0
Epoch 1/10
73/73 [==============================] - 24s 256ms/step - loss: 5.2254 - accuracy: 0.5549 - auc: 0.5167 - recall_5: 0.9091 - precision_5: 0.5051 - val_loss: 5.8363 - val_accuracy: 0.6389 - val_auc: 0.2735 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 2/10
73/73 [==============================] - 6s 80ms/step - loss: 4.4046 - accuracy: 0.4423 - auc: 0.3957 - recall_5: 0.4121 - precision_5: 0.3908 - val_loss: 4.8468 - val_accuracy: 0.6389 - val_auc: 0.2523 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 3/10
73/73 [==============================] - 6s 85ms/step - loss: 3.7309 - accuracy: 0.4670 - auc: 0.3859 - recall_5: 0.4121 - precision_5: 0.4121 - val_loss: 3.3988 - val_accuracy: 0.6389 - val_auc: 0.2715 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 4/10
73/73 [==============================] - 6s 86ms/step - loss: 3.1712 - accuracy: 0.4341 - auc: 0.3739 - recall_5: 0.3879 - precision_5: 0.3787 - val_loss: 2.6698 - va

## Count examples by label

In [18]:
stanford_ds = get_stratified_stanford_kfold_datasets('pet')[0] # pet, chest_ct, ct
santamaria_ds = get_stratified_santamaria_kfold_datasets('pet')[0]

stanford_label_counts = {0: 0, 1: 0}
santa_maria_label_counts = {0: 0, 1:0}

# Iterate through the dataset and count labels
for sample in stanford_ds[0]:
    label = sample[1].numpy()  # Assuming the label is a tensor
    stanford_label_counts[label] += 1


for sample in santamaria_ds[0]:
    label = sample[1].numpy()  # Assuming the label is a tensor
    santa_maria_label_counts[label] += 1

# Print the counts
print("Stanford Label 0 count:", label_counts[0])
print("Stanford Label 1 count:", label_counts[1])

print()
# Print the counts
print("Santa Maria Label 0 count:", santa_maria_label_counts[0])
print("Santa Maria Label 1 count:", santa_maria_label_counts[1])

print("--------------------------")
stanford_label_counts = {0: 0, 1: 0}
santa_maria_label_counts = {0: 0, 1:0}

# Iterate through the dataset and count labels
for sample in stanford_ds[1]:
    label = sample[1].numpy()  # Assuming the label is a tensor
    stanford_label_counts[label] += 1


for sample in santamaria_ds[1]:
    label = sample[1].numpy()  # Assuming the label is a tensor
    santa_maria_label_counts[label] += 1

# Print the counts
print("Testing Stanford Label 0 count:", label_counts[0])
print("Testing Stanford Label 1 count:", label_counts[1])

print()
# Print the counts
print("Testing Santa Maria Label 0 count:", santa_maria_label_counts[0])
print("Testing Santa Maria Label 1 count:", santa_maria_label_counts[1])


Stanford Label 0 count: 282
Stanford Label 1 count: 37

Santa Maria Label 0 count: 197
Santa Maria Label 1 count: 119
--------------------------
Testing Stanford Label 0 count: 282
Testing Stanford Label 1 count: 37

Testing Santa Maria Label 0 count: 30
Testing Santa Maria Label 1 count: 26
